# Geração Aumentada por Recuperação (RAG)

*Retrieval-Augmented Generation* (RAG) é uma técnica que "enriquece" o contexto fornecido a um Modelo de Linguagem de Grande Escala (LLM) antes do processamento do prompt. Essa abordagem permite que o modelo se concentre nas informações apresentadas, gerando respostas mais precisas e contextualmente relevantes.

Um exemplo prático de RAG é o [NotebookLM](https://notebooklm.google), uma ferramenta do Google projetada para analisar grandes quantidades de texto em arquivos, facilitando a extração de insights ([veja nosso vídeo](https://youtu.be/9Ab5iXh_5c8) sobre isso).

## Arquitetura RAG

De forma resumida, o processo começa com a extração de uma "consulta" (*query*) do prompt inicial. Esta consulta é então utilizada para realizar uma busca em um banco de dados vetorial, que atua como a "fonte de conhecimento", de onde são extraídas informações relevantes para enriquecer o contexto.

Após a recuperação e integração dessas informações adicionais, o prompt enriquecido é enviado ao LLM, que gera a resposta final. O diagrama abaixo ilustra essa arquitetura de forma visual.

![A imagem apresenta um diagrama de fluxo que descreve um processo de interação com um modelo de linguagem de grande escala (LLM).](https://docs.aws.amazon.com/images/sagemaker/latest/dg/images/jumpstart/jumpstart-fm-rag.jpg)

[Fonte da imagem](https://aws.amazon.com/pt/what-is/retrieval-augmented-generation/)

In [7]:
BOLD = "\033[1m"; RESET = "\033[0m"; GREEN = "\033[32m"; BLUE = "\033[34m"


## Fontes de conhecimento

Para cada tipo de aplicação, é desejável utilizar a arquitetura de banco de dados que melhor representa os dados do problema. Neste caso, em que o objetivo é recuperar informações via contexto, o melhor é utilizar um banco de dados vetorial, como o [ChromaDB](https://trychroma.com). Esses bancos de dados armazenam as informações na forma de vetores, de forma que vetores que descrevem temas correlacionados estão próximos (em termos da distância euclidiana) entre si.

In [8]:
from pathlib import Path

datasource = list(Path("./data/aws-case-studies-and-blogs/files").glob("**/*.txt"))
print(f"Encontrei {len(datasource)} arquivos.")

Encontrei 347 arquivos.


In [9]:
import chromadb

# Create a new database
client = chromadb.PersistentClient(path="chromadb")
coll = client.get_or_create_collection("aws-case-studies-and-blogs")

In [10]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
import tqdm

text_splitter = SemanticChunker(OpenAIEmbeddings())



files = []

for path in datasource:
    content = path.read_text()
    files.append(content)

print(f"Li {len(files)} arquivos.")

Li 347 arquivos.


## Salvando no Banco de Dados
A célula a seguir vai armazenar os documentos carregados no Chroma DB. Na minha máquina, para 347 arquivos, levou 7 min (x documentos).

In [11]:
count = coll.count()
print(f"A coleção já contém {count} documentos.")


batch_size = 30
# Load the documents in batches of batch_size
for i in tqdm.tqdm(
    range(0, len(files), batch_size), desc="Adicionando documentos", unit_scale=batch_size
):

    docs = text_splitter.create_documents(files[i : i + batch_size])
    ids = [f"id{i}-{j}" for j in range(len(docs))]
    coll.add(
        ids=ids,
        documents=[d.page_content for d in docs],
    )

new_count = coll.count()
print(f"Adicionei {new_count - count} documentos")

Collection already contains 0 documents


Adding documents: 100%|██████████| 360/360 [05:53<00:00,  1.02it/s]

Added 1477 documents


## Fazendo pesquisas na base de dados

Agora que já populamos a base de dados, podemos fazer perguntas especializadas.

In [ ]:
search = coll.query(
    query_texts=["interpretação de argumentos de linha de comando"],
    n_results=10,
)

search["documents"][0]

O resultado com a docs do Python não foi satisfatório.